#This notebook is for the capstone project


In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 5.7 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=e7380a701557f61b1596c6a98ac5171668ca7b5bb0727787169bb545b7906f79
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


In [5]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [6]:
# CODE FOR GOOGLE https://www.google.com/?code=C5C1IHXQHPR5VS3OUOVBBAE2X0J141KHZ1VYDCRXWIA3PDPJ#_=_

Your credentails:
CLIENT_ID: OK1PKC5SF4YLBOMHW2DAHZGWPDARF0MLETMAEC4PDHHCBK2Z
CLIENT_SECRET:NF5MXY1530ZQUGSKEWYHTK2YH0BU0V54G1XUBJCLMXBYY0DS


# Importing the data from the wikipedia page trough beautifoulsoup

In [43]:
import requests 

from bs4 import BeautifulSoup 

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 

soup = BeautifulSoup(req.content,'lxml') 

table = soup.find_all('table')[0]  

df = pd.read_html(str(table)) 

df_toronto=pd.DataFrame(df[0])
df_toronto.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [44]:
# Remove not assigned values on borough

df_toronto_borough = df_toronto[df_toronto['Borough'] != 'Not assigned']
df_toronto_borough.head()

# Regrouping by postal code is done previously 

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [45]:
# If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

df_toronto_neighbourhood = df_toronto_borough[df_toronto_borough['Neighbourhood'] == 'Not assigned']
df_toronto_neighbourhood.head()

,Postal Code,Borough,Neighbourhood


There is no neighbourhood with a not assigned value so we are keeping the df_toronto_borough dataframe


In [48]:
df_toronto_borough.shape

(103, 3)

## Once we have these information we can go to the next step of the assigment

In [56]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [59]:
#We have now two dataframe df_geospatial_coordinates & df_toronto_borough. We have to join them to be able to have a final dataframe

df_merged = pd.merge(df_toronto_borough, df_geospatial_coordinates, on='Postal Code')
df_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [61]:
# The dataframes are now merged on the postal code